# Human-in-the-Loop with LangChain 1.0

## Expense Approval System

This notebook shows how to use **LangChain 1.0** `HumanInTheLoopMiddleware` for approval workflows.

### Key Concepts:
- Use `create_agent()` with middleware
- Configure which tools need approval
- Three response types: accept, edit, respond

---

## 1. Setup

In [ ]:
# Install packages
!pip install -qU langchain langchain-openai langgraph

In [ ]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

print("✅ Setup complete!")

---

## 2. Define Tools

Create tools for expense operations:

In [ ]:
from langchain_core.tools import tool

@tool
def process_expense(employee_name: str, amount: float, description: str) -> str:
    """Process and approve an expense for reimbursement.
    
    Args:
        employee_name: Name of the employee
        amount: Expense amount in INR
        description: Description of the expense
    """
    return (
        f"✅ Expense Processed\n"
        f"Employee: {employee_name}\n"
        f"Amount: ₹{amount:,.2f}\n"
        f"Description: {description}\n"
        f"Status: Approved for reimbursement"
    )

@tool
def send_notification(employee_name: str, message: str) -> str:
    """Send a notification to an employee.
    
    Args:
        employee_name: Name of the employee
        message: Notification message
    """
    return f"📧 Notification sent to {employee_name}: {message}"

tools = [process_expense, send_notification]

print("✅ Tools defined")
print(f"   - {tools[0].name}: Requires approval")
print(f"   - {tools[1].name}: Auto-approved")

---

## 3. Create Agent with HITL Middleware

This is the key part - configure which tools need approval:

In [ ]:
from langchain import create_agent, HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import MemorySaver

# Configure HITL middleware
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        # process_expense requires approval with all response types
        "process_expense": ["accept", "edit", "respond"],
        
        # send_notification doesn't need approval (not listed)
    }
)

# Create checkpointer (required for HITL)
checkpointer = MemorySaver()

# Create agent with middleware
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    middleware=[hitl_middleware],
    checkpointer=checkpointer,
)

print("✅ Agent created with HITL middleware")
print("\n   Approval Policy:")
print("   🛑 process_expense → Requires approval")
print("   ▶️  send_notification → Auto-approved")

---

## 4. Example 1: Submit Expense (Requires Approval)

### Step 1: Submit the expense

In [ ]:
print("=" * 60)
print("EXAMPLE 1: EXPENSE APPROVAL WORKFLOW")
print("=" * 60)
print()

# Thread configuration
config = {"configurable": {"thread_id": "expense-001"}}

# User request
user_message = (
    "Please process an expense for Priya Sharma: "
    "₹5000 for client dinner at Taj Hotel, Mumbai"
)

print("📝 User Request:")
print(f"   {user_message}")
print()

# Invoke agent
result = agent.invoke(
    {"messages": [{"role": "user", "content": user_message}]},
    config=config
)

print("\n" + "=" * 60)

### Step 2: Check if paused for approval

In [ ]:
if "__interrupt__" in result:
    print("\n🛑 EXECUTION PAUSED - APPROVAL REQUIRED")
    print("=" * 60)
    print()
    
    # Get interrupt details
    interrupts = result["__interrupt__"]
    
    for interrupt in interrupts:
        action_requests = interrupt.get("value", {}).get("action_requests", [])
        
        for action in action_requests:
            print(f"Tool: {action['name']}")
            print(f"Arguments:")
            for key, val in action.get('arguments', {}).items():
                print(f"  {key}: {val}")
            print()
    
    print("💡 Manager needs to review and approve...")
    print("\nAllowed responses: accept, edit, respond")
else:
    print("✅ Executed without approval (tool doesn't require it)")

### Step 3: Manager Approves (Option A)

In [ ]:
from langgraph.types import Command

print("=" * 60)
print("OPTION A: ACCEPT AS-IS")
print("=" * 60)
print()

# Manager approves
approval_response = {
    "decisions": [
        {
            "type": "accept"
        }
    ]
}

print("👨‍💼 Manager Decision: ACCEPT")
print("   Action: Approve expense as-is")
print()

# Resume execution
result = agent.invoke(
    Command(resume=approval_response),
    config=config
)

print("\n" + "=" * 60)
print("FINAL RESULT")
print("=" * 60)
print()

# Print the final message
if "messages" in result:
    last_message = result["messages"][-1]
    print(last_message.content if hasattr(last_message, 'content') else last_message)

### Alternative: Manager Edits (Option B)

In [ ]:
# To use this, run Example 1 again with a new thread_id, then use this:

print("=" * 60)
print("OPTION B: EDIT BEFORE APPROVAL")
print("=" * 60)
print()

# Manager modifies the amount
edit_response = {
    "decisions": [
        {
            "type": "edit",
            "args": {
                "employee_name": "Priya Sharma",
                "amount": 4500.0,  # Reduced from 5000
                "description": "Client dinner at Taj Hotel, Mumbai (adjusted)"
            }
        }
    ]
}

print("👨‍💼 Manager Decision: EDIT")
print("   Change: Reduced amount from ₹5000 to ₹4500")
print("   Reason: Removed alcohol from bill")
print()
print("# To use: agent.invoke(Command(resume=edit_response), config)")

### Alternative: Manager Rejects (Option C)

In [ ]:
print("=" * 60)
print("OPTION C: REJECT WITH FEEDBACK")
print("=" * 60)
print()

# Manager rejects
reject_response = {
    "decisions": [
        {
            "type": "respond",
            "message": "Expense rejected. Client dinners over ₹3000 require prior approval from department head."
        }
    ]
}

print("👨‍💼 Manager Decision: REJECT")
print("   Reason: Exceeds approval limit")
print("   Feedback: Requires escalation to department head")
print()
print("# To use: agent.invoke(Command(resume=reject_response), config)")

---

## 5. Example 2: Send Notification (Auto-Approved)

Tools not in `interrupt_on` execute immediately:

In [ ]:
print("=" * 60)
print("EXAMPLE 2: AUTO-APPROVED ACTION")
print("=" * 60)
print()

# New thread
config_2 = {"configurable": {"thread_id": "notify-001"}}

# Request notification
notification_message = "Send a notification to Rahul Verma saying his expense is pending review"

print("📝 Request:")
print(f"   {notification_message}")
print()

result = agent.invoke(
    {"messages": [{"role": "user", "content": notification_message}]},
    config=config_2
)

print("\n" + "=" * 60)

if "__interrupt__" in result:
    print("🛑 Paused (unexpected!)")
else:
    print("✅ EXECUTED IMMEDIATELY (No approval needed)")
    print("=" * 60)
    print()
    if "messages" in result:
        last_message = result["messages"][-1]
        print(last_message.content if hasattr(last_message, 'content') else last_message)
    print()
    print("💡 Why no approval?")
    print("   • 'send_notification' is not in interrupt_on config")
    print("   • Low-risk operation")
    print("   • Can be easily reversed")

---

## 6. How It Works

### Key Components:

```python
# 1. Create middleware
hitl_middleware = HumanInTheLoopMiddleware(
    interrupt_on={
        "tool_name": ["accept", "edit", "respond"]
    }
)

# 2. Add to agent
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    middleware=[hitl_middleware],
    checkpointer=checkpointer
)

# 3. Invoke - will pause if tool matches
result = agent.invoke(messages, config)

# 4. Resume with decision
agent.invoke(
    Command(resume={"decisions": [{"type": "accept"}]}),
    config
)
```

### Three Response Types:

| Type | Action | Use Case |
|------|--------|----------|
| `accept` | Execute as-is | Approve without changes |
| `edit` | Modify args | Change amount before processing |
| `respond` | Reject with feedback | Decline and explain why |

---

## 7. Production Tips

In [ ]:
print("💡 Production Recommendations:")
print()
print("1. Use PostgreSQL Checkpointer:")
print("   from langgraph.checkpoint.postgres import PostgresSaver")
print("   checkpointer = PostgresSaver.from_conn_string(...)")
print()
print("2. Add Notifications:")
print("   • Send Slack message when approval needed")
print("   • Email stakeholders with approval link")
print("   • Create ticket in approval system")
print()
print("3. Build Approval Dashboard:")
print("   • Web UI showing pending approvals")
print("   • Approve/reject/edit from browser")
print("   • Track approval history")
print()
print("4. Configure by Amount/Role:")
print("   • Different approval thresholds")
print("   • Multi-level approvals for large amounts")
print("   • Role-based approval routing")

---

## Summary

### ✅ What We Built:
- Simple expense approval system
- LangChain agent with HITL middleware
- Selective tool approval
- Three response types (accept/edit/respond)

### 🔑 Key Differences from LangGraph:

| Feature | LangChain | LangGraph |
|---------|-----------|------------|
| API | `create_agent()` + middleware | `StateGraph` + `interrupt()` |
| HITL | `HumanInTheLoopMiddleware` | `interrupt()` function |
| Config | `interrupt_on` dict | Manual in nodes |
| Best For | Tool approval | Complex workflows |

### 📚 Use This When:
- ✅ Need tool call approval
- ✅ Simple agent workflows
- ✅ Built-in accept/edit/respond

### 📚 Use LangGraph When:
- ✅ Complex multi-step workflows
- ✅ Custom approval logic
- ✅ Multiple approval gates

---

**End of Notebook**